# Below program is to extract data from PDF and store it in an Excel file

### Load necessary libraries

In [1]:
import pdftotext
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

### Load PDF

In [2]:
# Load your PDF
with open(r"C:\Users\admin\Documents\Canopy Interview\Python Script\test_input.pdf", "rb") as f:
    pdf = pdftotext.PDF(f)

### Write PDF to a temporary text file

In [3]:
# write to a text file
with open(r'C:\Users\admin\Documents\Canopy Interview\Python Script\test_outputt.txt', 'w') as f:
    f.write("\n".join(pdf))

### Extract data from the temp text file and create a list

In [4]:
# Extract data from the temp file and create a list
with open(r'C:\Users\admin\Documents\Canopy Interview\Python Script\test_outputt.txt') as f:
    lines = [line.rstrip() for line in f]

### Remove blanks from lists and verify the length

In [5]:
str_list = filter(lambda item: item, lines)
str_list = list(filter(None, str_list))
len(str_list)

33

### Cleanse the list with appropriate split. (e.g. Here I have used double space to split each words as an item)

In [6]:
new_list = [i.split('  ') for i in str_list]
r_list = []
for i in range(len(new_list)):
    r_str = list(filter(None, new_list[i]))
    r_list.append(r_str)

### Convert the list to a DataFrame

In [7]:
df = pd.DataFrame(data=r_list)
df.columns = ['Value Date','Order Date','Description','Amount']

### Extract values with in the paranthesis and create a new column "Debit"

In [8]:
df["Debit"] = df["Amount"].str.extract(r"\((.*?)\)", expand=False)

### Create a new column "Currency" 

In [9]:
df['Currency'] = pd.np.where(df['Value Date'].str.contains("AUD"), "AUD",
                               pd.np.where(df['Value Date'].str.contains("HKD"), "HKD",
                               pd.np.where(df['Value Date'].str.contains("USD"), "USD","")))
df['Currency'] = df['Currency'].replace('', np.nan).ffill()
df = df.replace(np.nan,'')

### Remove values of paranthesis in Amount

In [10]:
df['Amount'] = np.where(df['Amount'].str.contains(r'[()]'),"",df['Amount'])

### Merge Description as few items has multiline entries

In [11]:
df['Description'] = np.where(df['Value Date'].str.contains('ISIN'), df['Value Date'],df['Description'])
search_string = 'ISIN'
for i in range(len(df['Description'])):
    if search_string in df['Value Date'].iloc[i]:
        df['Description'].iloc[i-1] = ' '.join([df['Description'].iloc[i-1], df['Description'].iloc[i]])

### Filter records which has date values in 'Value Date' Column

In [12]:
df['Value Date'] = np.where(df['Value Date'].str.match(r'^\d{2} [a-zA-Z]{3} \d{4}$'),df['Value Date'],"")
df['Value Date'] = df['Value Date'].replace('',np.nan)
df = df[df['Value Date'].notna()]

### Rename and Update Column names, Order

In [13]:
df = df.rename(columns={"Amount":"Credit"})
df = df[['Value Date', 'Order Date', 'Description', 'Currency', 'Debit','Credit']]

### Change formats of Date, Credit and Debit values. Sort the records by 'Value Date'

In [14]:
df['Value Date'] = pd.to_datetime(df['Value Date'])
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Value Date'] = df['Value Date'].dt.strftime('%Y-%m-%d')
df['Order Date'] = df['Order Date'].dt.strftime('%Y-%m-%d')
df.sort_values(by='Value Date')
df['Debit'] = df['Debit'].str.replace(',', '')
df['Credit'] = df['Credit'].str.replace(',', '')

In [15]:
df.reset_index()
df.reset_index(drop=True)
df.set_index("Value Date", inplace = True)
df

,Order Date,Description,Currency,Debit,Credit
Value Date,,,,,
2019-03-01,2019-02-28,INTEREST RECEIVED,HKD,,2.39
2019-03-04,2019-02-27,INTEREST FOR BNP PARIBAS 4M CALLABLE FIXED CO...,HKD,,20025.00
2019-03-04,2019-03-04,"EARLY REDEMPTION OF HKD 3,000,000.00 BNP PARI...",HKD,,3000000.00
2019-03-05,2019-03-05,REPAY FIXED LOAN #31806-0,HKD,5443428.95,
2019-03-05,2019-03-05,INTEREST CHARGE ON FIXED LOAN #31806-0,HKD,1834.73,
2019-03-05,2019-03-05,DRAWDOWN FIXED LOAN #38034-0,HKD,,5445263.68
2019-03-12,2019-03-12,REPAY FIXED LOAN #38034-0,HKD,5445263.68,
2019-03-12,2019-03-12,INTEREST CHARGE ON FIXED LOAN #38034-0,HKD,1998.34,
2019-03-12,2019-03-12,DRAWDOWN FIXED LOAN #43286-0,HKD,,2427234.63


### Save the dataframe to an excel file

In [16]:
df.to_excel("PDF2Excel.xlsx")